In [2]:
import os

predicted_binding_sites_folder = 'predicted_binding_sites'

prediction_files = list(filter(lambda file: '_predictions' in file , os.listdir(predicted_binding_sites_folder)))

print(len(prediction_files))

49


In [86]:
import pandas as pd
import re

df = pd.read_csv(os.path.join(predicted_binding_sites_folder, prediction_files[0]))

#df.iloc[0,9].strip().split(' ') # list

#[token.split('_')[-1] for token in df.iloc[0,9].strip().split(' ')] #residue ids


In [33]:
binding_residues = set()
for binding_pred_file in prediction_files:
    df = pd.read_csv(os.path.join(predicted_binding_sites_folder, binding_pred_file))
    #print(df.iloc[0,9].strip())
    #print(type(df.iloc[0,9].strip()))
    for token in df.iloc[0,9].strip().split(' '):
        binding_residues.add(token)
binding_residues #binding site residues which are predicted, an unique set

{'A_105',
 'A_106',
 'A_107',
 'A_108',
 'A_117',
 'A_118',
 'A_119',
 'A_120',
 'A_126',
 'A_137',
 'A_173',
 'A_180',
 'A_183',
 'A_184',
 'A_188',
 'A_211',
 'A_212',
 'A_213',
 'A_215',
 'A_216',
 'A_220',
 'A_221',
 'A_223',
 'A_224',
 'A_227',
 'A_230',
 'A_241',
 'A_271',
 'A_299',
 'A_301',
 'A_302',
 'A_303',
 'A_304',
 'A_305',
 'A_306',
 'A_308',
 'A_309',
 'A_310',
 'A_311',
 'A_312',
 'A_313',
 'A_315',
 'A_364',
 'A_369',
 'A_370',
 'A_371',
 'A_372',
 'A_373',
 'A_374',
 'A_375',
 'A_398',
 'A_433',
 'A_434',
 'A_435',
 'A_436',
 'A_437',
 'A_440',
 'A_441',
 'A_442',
 'A_443',
 'A_444',
 'A_447',
 'A_448',
 'A_451',
 'A_481',
 'A_482',
 'A_483',
 'A_484',
 'A_485',
 'A_488',
 'A_50',
 'A_57',
 'A_76',
 'A_78',
 'A_79',
 'A_94'}

In [85]:
s = set()
for binding_pred_file in prediction_files:
    df = pd.read_csv(os.path.join(predicted_binding_sites_folder, binding_pred_file))
    #print(df.iloc[0,9].strip())
    #print(type(df.iloc[0,9].strip()))
    lst = []
    #print(df.iloc[0,9].strip().split(' '))
    
    for token in df.iloc[0,9].strip().split(' '):
        lst.append(token)
        
    s.add(''.join(lst))
    
len(s)

13

In [87]:
len(binding_residues)

76

In [139]:
def compute_centroid(vectors):
    """
        vectors: a list of lists for eg: [[1,2,3], [4,5,6], [7,8,9]]
        note that each inner list is a list of float values
        and a inner list is a row vector in R^3 for eg: given, [1,2,3], 1 is the x-coordinate, 2 is the y-coordinate and 3 is the z-coordinate
        
        returns a  centroid as a row_vector in R^3
    """
    import numpy as np
    #we are going to represent vectors as a 2d matrix, with each row being a row vector in R^3
    two_d_matrix = np.array(vectors)
    n = len(vectors)
    centroid = 1/n * np.sum(two_d_matrix, axis=0)
    return centroid

def get_residue_ids_from_p2rank_predictions(pred_csv):
    """
        pred_csv (str): abs file path to a prediction.csv generated by p2rank
        return a sorted list of residues_id, for eg ['A_1', 'A_2'...]
    """
    import pandas as pd
    
    pred_result_df = pd.read_csv(pred_csv)
    #going to get the top result, a.k.a first row
    
    return sorted(pred_result_df.iloc[0, 9].strip().split(' '), key=lambda token: int(token.split('_')[-1]))

def get_mutant_code_from_pred_csv(pred_csv):
    
    """
        pred_csv (str): for this function, don't need to get abs file path.
        for eg: CYP3A4.3.pdbqt_predictions.csv
        
        return just the mutant code, for eg: CYP3A4.3
    """
    return pred_csv.split('_')[0].replace('.pdbqt', '')

def get_pdb_with_mutant_code(folder, mutant_code):
    """
        folder (str): path to folder which stores pdbs generated by scwrl
        eg: 'pdb_f/scwrl_out'
        mutant_code (str): mutant
    """
    
    mutant_pdbs = os.listdir(folder)
    
    return list(filter(lambda file: mutant_code in file, mutant_pdbs))[0]

def get_vectors_of_residue_r_group(residue_ids, mutant_code, pdb_folder):
    """
        residue_ids (list of str): for eg: ['A_50','A_57','A_76','A_78'], it must be sorted
        mutant_code (str): for eg: CYP3A4.11
        pdb_folder (str): for eg: 'pdb_f/scwrl_out'
        
        return map for eg 'A_50': [vector_of_atom1_res_50, vector_of_atom2_res50] etc

    """
    #get correct pdb
    pdb_files = os.listdir(pdb_folder)
    mutant_pdb = list(filter(lambda file: mutant_code in file, pdb_files))[0]
    
    import re
    pattern = re.compile('^ATOM')
    back_bone_atoms = set(['N', 'CA', 'O', 'C'])
    
    result = {}
    
    with open(pdb_folder + '/' + mutant_pdb, 'r') as mutant_pdb:
        lines = mutant_pdb.readlines()
        
        for res_id in residue_ids:
            chain, res_num = res_id.split('_')
            result[res_id] = []
            
            for line in lines:
                if pattern.match(line):
                    tokens = re.split("\s{1,}", line)
                    
                    from_pdb_chain_type = tokens[4]
                    from_pdb_res_number = tokens[5]
                    from_pdb_atom = tokens[2]
                    
                    if chain == from_pdb_chain_type and res_num == from_pdb_res_number and not from_pdb_atom in back_bone_atoms:
                        print(from_pdb_atom)
                        x_coord = tokens[6]
                        y_coord = tokens[7]
                        z_coord = tokens[8]
                        
                        result[res_id].append([x_coord, y_coord, z_coord])
                        
                         
    return result  

In [140]:
a = np.array([[1,2,3], [4,5,6], [7,8,9]])

1/3 * np.sum(a, axis=0)

array([4., 5., 6.])

In [141]:
compute_centroid([[1,2,3], [4,5,6], [7,8,9]])

array([4., 5., 6.])

In [142]:
res_ids = get_residue_ids_from_p2rank_predictions('predicted_binding_sites/CYP3A4.3.pdbqt_predictions.csv')

In [143]:
mutant_code = get_mutant_code_from_pred_csv('CYP3A4.3.pdbqt_predictions.csv')

In [144]:
get_vectors_of_residue_r_group(res_ids, mutant_code, 'pdb_f/scwrl_out')

CB
CD1
CG1
CG2
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
CG
OD1
OD2
CB
CD
CG
OE1
NE2
CB
CD
CG
OE1
NE2
CB
CG
CD1
CD2
CB
CD
NE
CG
CZ
NH1
NH2
CB
CD
NE
CG
CZ
NH1
NH2
CB
CD
CG
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
CD1
CG1
CG2
CB
OG
CB
CD1
CG1
CG2
CB
CG
CD1
NE1
CD2
CE2
CH2
CZ2
CE3
CZ3
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
OG
CB
CG1
CG2
CB
CD1
CG1
CG2
CB
OG
CB
CD
NE
CG
CZ
NH1
NH2
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
CG
CD1
CD2
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
CG
CD1
CD2
CB
CD1
CG1
CG2
CB
OG1
CG2
CB
CD
CG
CB
CD1
CG1
CG2
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
OG
CB
CD1
CG1
CG2
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
CD1
CG1
CG2
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
CB
OG1
CG2
CB
OG1
CG2
CB
OG
CB
CG1
CG2
CB
CG
CD1
CD2
CB
CD1
CG1
CG2
CB
CB
SD
CE
CG
CB
CD
NE
CG
CZ
NH1
NH2
CB
CG
CD1
CD2
CB
CD
CG
OE1
OE2
CB
CD
NE
CG
CZ
NH1
NH2
CB
CD
CG
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
CD
NE
CG
CZ
NH1
NH2
CB
CG
OD1
ND2
CB
SG
CB
CD1
CG1
CG2
CB
CG
CZ
CD1
CE1
CD2
CE2
CB
CB
CG
OD1
ND2
CB
CG
CD1
CD2
CB
CG
CD1
CD2


{'A_50': [['-32.926', '-5.614', '-19.154'],
  ['-34.962', '-5.058', '-17.724'],
  ['-34.209', '-4.793', '-19.009'],
  ['-31.891', '-5.239', '-18.105']],
 'A_57': [['-26.206', '-13.537', '-21.543'],
  ['-26.801', '-12.962', '-20.289'],
  ['-27.902', '-11.892', '-17.972'],
  ['-27.420', '-13.781', '-19.361'],
  ['-27.968', '-13.252', '-18.208'],
  ['-26.743', '-11.603', '-20.038'],
  ['-27.291', '-11.074', '-18.885']],
 'A_76': [['-27.467', '-4.771', '-16.324'],
  ['-26.812', '-5.614', '-17.401'],
  ['-27.306', '-5.601', '-18.548'],
  ['-25.805', '-6.287', '-17.098']],
 'A_78': [['-29.003', '2.616', '-13.060'],
  ['-30.747', '3.958', '-14.300'],
  ['-30.445', '3.095', '-13.091'],
  ['-30.134', '5.008', '-14.494'],
  ['-31.694', '3.517', '-15.120']],
 'A_79': [['-25.432', '-1.820', '-13.393'],
  ['-23.174', '-1.691', '-12.269'],
  ['-24.044', '-2.426', '-13.269'],
  ['-23.542', '-1.539', '-11.104'],
  ['-22.013', '-1.230', '-12.722']],
 'A_94': [['-8.499', '-11.757', '-16.138'],
  ['-9.89